In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import easyocr
import cv2

In [2]:
DATADIR="img"
CATEGORIES=["ID_CARD","FOTOKOPI"]
IMG_SIZE=100
training_data=[]

def create_training_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIR, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass

create_training_data()

In [3]:
print(len(training_data))

2743


In [5]:
import random
random.shuffle(training_data)

In [6]:
x=[]
y=[]
for features, label in training_data:
    x.append(features)
    y.append(label)
x=np.array(x).reshape(-1,IMG_SIZE, IMG_SIZE,3)

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle

In [8]:
x=x/255.0

y=np.array(y)

In [9]:
model=Sequential()
model.add(Conv2D(64, (3,3), input_shape=x.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss="binary_crossentropy", 
              optimizer="adam", 
              metrics=['accuracy'])
model.fit(x, y, batch_size=8, epochs=12, validation_split=0.3)

Epoch 1/12
240/240 [==============================] - 40s 158ms/step - loss: 0.2466 - accuracy: 0.9417 - val_loss: 0.0425 - val_accuracy: 0.9866
Epoch 2/12
240/240 [==============================] - 37s 155ms/step - loss: 0.0664 - accuracy: 0.9818 - val_loss: 0.0749 - val_accuracy: 0.9635
Epoch 3/12
240/240 [==============================] - 37s 155ms/step - loss: 0.0682 - accuracy: 0.9792 - val_loss: 0.0516 - val_accuracy: 0.9806
Epoch 4/12
240/240 [==============================] - 37s 156ms/step - loss: 0.0653 - accuracy: 0.9854 - val_loss: 0.0231 - val_accuracy: 0.9988
Epoch 5/12
240/240 [==============================] - 38s 157ms/step - loss: 0.0144 - accuracy: 0.9974 - val_loss: 0.0084 - val_accuracy: 0.9964
Epoch 6/12
240/240 [==============================] - 38s 157ms/step - loss: 2.0513e-04 - accuracy: 1.0000 - val_loss: 1.8217e-04 - val_accuracy: 1.0000
Epoch 7/12
240/240 [==============================] - 38s 156ms/step - loss: 0.1990 - accuracy: 0.9495 - val_loss: 0.0503 

In [10]:
CATEGORIES=["ID_CARD","FOTOKOPI"]
def prepare(filepath):
    IMG_SIZE=100
    img_array=cv2.imread(filepath)
    new_array=cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
prediction=model.predict([prepare("FOTOKOPI/_234201.jpg")])
print(CATEGORIES[int(prediction[0][0])])

1/1 [==============================] - 0s 289ms/step
FOTOKOPI


In [11]:
CATEGORIES=["ID_CARD","FOTOKOPI"]
def prepare(filepath):
    IMG_SIZE=100
    img_array=cv2.imread(filepath)
    new_array=cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
prediction=model.predict([prepare("TEST_ID_CARD/_090835.jpg")])
print(CATEGORIES[int(prediction[0][0])])

1/1 [==============================] - 0s 56ms/step
ID_CARD


In [12]:
from joblib import Parallel, delayed
import joblib

joblib.dump(model, 'model.pkl')

['model.pkl']

In [14]:
benim = joblib.load('model.pkl')

In [15]:
CATEGORIES=["ID_CARD","FOTOKOPI"]
def prepare(filepath):
    IMG_SIZE=100
    img_array=cv2.imread(filepath)
    new_array=cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
prediction=benim.predict([prepare("TEST_ID_CARD/_090835.jpg")])
print(CATEGORIES[int(prediction[0][0])])

1/1 [==============================] - 0s 155ms/step
ID_CARD
